In [1]:
## Import packages + libraries for later use
%matplotlib inline

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import preprocessing

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import load_img,img_to_array
from keras.models import Model

Using TensorFlow backend.


In [2]:
## Import specific modules needed for CNN

from VisualiseFilters import SaliencyMask, GradientSaliency, VisualBackprop, deprocess_image, vis_img_in_filter, get_layer_names, activation_maps
from Predictions import make_predictions, show_predictions
from ViewHistory import view_history
from GetImages import get_images
from Shuffling import stratified_shuffling

In [3]:
## Dataset which will be learned
data = 'disp'
GRAY=True
## Desired resolution of images e.g. (28x28)
img_res = 227

In [4]:
## File directory to jpg folder
train_dir = 'C:/Users/barry.odonnell/Python/input_eye/train_jpg_files_{}'.format(data)

## Collects training images and test images
train_imgs = ['C:/Users/barry.odonnell/Python/input_eye/train_jpg_files_{}/{}'.format(data, i) for i in os.listdir(train_dir)]

In [5]:
if data == 'disp':
    data = 'disp-short'

In [6]:
## Obtain image and label arrays
X,y = get_images(train_imgs, img_res, GRAY=GRAY, data=data)

In [7]:
## create an array of each value in the set and assign them integers from 0-36 
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)

print("Shape of train images is:", X.shape) ## Check for correct shape, should have 4 axes
print("Shape of labels is:       ({}, {})".format(len(Y), max(Y) + 1))

y = np.array(y)
lb = preprocessing.LabelBinarizer()
lb.fit(Y)

Shape of train images is: (1547, 227, 227, 1)
Shape of labels is:       (1547, 10)


LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [8]:
X_train, Y_train, X_val, Y_val = stratified_shuffling(X, Y, val_percent=0.3)

Stratified shuffling...
Finish stratified shuffling...


In [12]:
## Begin definition for Neural Network model.
##  A Convolutional Neural Network (CNN) is used for its
##  versatility with images, and it's feature map ability.
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

## Insert image dimensions, preferably square, includinig channel
## Number of classes
img_dim = (img_res,img_res,1)
n_classes = max(Y) + 1

model_name = 'eye_keras_cnn-model_{}_{}.h5'.format(data, img_res)
#model_name = 'model.h5'
## Checks if this model exists.
##  If it does, loads said model and skips model building.
if os.path.exists(model_name):
    print("Loading model...")
    model = load_model(model_name)
    print("{} has been loaded...".format(model_name))
    
## Else, builds 'Sequential' model and will contain the layers stated below 
##  https://keras.io/layers/core/
##  https://keras.io/layers/convolutional/
else:
    print("Building Model...")

    model = Sequential()

    model.add(Conv2D(8, (11,11), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (4,4)))

    model.add(Conv2D(64, (9,9), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))

    model.add(Conv2D(16, (9,9), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (3,3)))
    
    model.add(Conv2D(16, (9,9), input_shape=(img_dim), kernel_initializer='normal'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (3,3)))
    
    model.add(Flatten())
    model.add(Dense(img_res**2))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    model.add(Dense(n_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', 
                  metrics=['accuracy']
                 )
    
    print("\nModel ('{}') built...".format(model_name))

## Returns model info for user to review and check that it is correct
model.summary()

Building Model...

Model ('eye_keras_cnn-model_disp-short_227.h5') built...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 217, 217, 8)       976       
_________________________________________________________________
activation_7 (Activation)    (None, 217, 217, 8)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 54, 54, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 46, 46, 64)        41536     
_________________________________________________________________
activation_8 (Activation)    (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 38, 38, 16)        82960     
_________________________________________________________________


In [13]:
# data augmentation
datagen = ImageDataGenerator(
    featurewise_center = False,
    samplewise_center = False,
    featurewise_std_normalization = False,
    samplewise_std_normalization = False,
    zca_whitening = False,
    rotation_range = 0,
    zoom_range = 0,
    width_shift_range = .3,
    height_shift_range = 0,
    horizontal_flip = True,
    vertical_flip = False
    )

datagen.fit(X_train)

In [ ]:
# Begin testing on model
print('Training model...')
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size = 32),
                   epochs = 25,
                   validation_data = (X_val, Y_val),
                   steps_per_epoch = 10,
                   verbose = 1
                   )

print('Validating model...')
score, acc = model.evaluate(X_val, Y_val, verbose= 1)
print('\nLoss:', score, '\nAcc:', acc)

Training model...
Epoch 1/25
10/10 [==============================] - 162s 16s/step - loss: 2.2916 - acc: 0.1094 - val_loss: 2.2297 - val_acc: 0.1419
Epoch 2/25
10/10 [==============================] - 145s 15s/step - loss: 2.1383 - acc: 0.1469 - val_loss: 2.0927 - val_acc: 0.2108
Epoch 3/25
10/10 [==============================] - 160s 16s/step - loss: 1.8736 - acc: 0.2687 - val_loss: 1.9283 - val_acc: 0.2301
Epoch 4/25
10/10 [==============================] - 140s 14s/step - loss: 1.6728 - acc: 0.3283 - val_loss: 1.4938 - val_acc: 0.4473
Epoch 5/25
10/10 [==============================] - 140s 14s/step - loss: 1.4206 - acc: 0.3906 - val_loss: 1.2919 - val_acc: 0.4753
Epoch 6/25
10/10 [==============================] - 138s 14s/step - loss: 1.1757 - acc: 0.5344 - val_loss: 1.1000 - val_acc: 0.5914
Epoch 7/25
10/10 [==============================] - 140s 14s/step - loss: 1.0470 - acc: 0.6059 - val_loss: 0.9220 - val_acc: 0.5763
Epoch 8/25
10/10 [==============================] - 145s 1

In [ ]:
## Save model
print("\nSaving model...")
model.save('{0:3.2f}_'.format(acc*100)+ model_name)
print("\nSaved model as {}...".format(model_name))

In [ ]:
view_history(history, show_acc = False)
view_history(history, show_acc = True)

In [ ]:
make_predictions(model=model, 
                 test_set=X_val, 
                 val_set=Y_val)

In [ ]:
layers = get_layer_names(model, show=True)

In [ ]:
vis_img_in_filter(model,
                  X_train,
                  img_num=-2,
                  img_res=img_res,
                  layer_num=8,
                  GRAY=GRAY
                 )

In [ ]:
kernel_sides = [11,9,9,9]
max_pool_sides = [4,,3]

In [ ]:
def mock_convolutional(layer_num=0, filter_num=0, conv_kernel_side=kernel_sides[0], img_res=img_res, image = 0, max_prev = False, max_pooled_img=None, max_pools_per_side=img_res):
    
    #plt.figure(figsize = (3,3))

    ## Flatten array to create continuous array
    if max_prev == True:
        img_res = max_pools_per_side
    
    
    kernel_side = conv_kernel_side
    kernel = kernel_side**2

    neg_res = (kernel_side//2) * 2
    new_img_res = img_res-neg_res
    
    
    layer = model.layers[layer_num]
    conv_weights = layer.get_weights()[0][:,:,0,:]
    
    concat_weights = np.concatenate(conv_weights[:,:,filter_num])
    
    convolve_flat = (kernel)*((new_img_res)**2)  ## works out size needed for flattened filter

    ## Create filter array
    filtered = np.zeros(convolve_flat)#np.zeros(img_res**2)

    ## Begins loop to determine place so flat multiplication can be done
    num = 0
    for j in range(new_img_res):
        for i in range(new_img_res):
            for y in range(kernel_side):
                for x in range(kernel_side):
                    if max_prev == False:
                        filtered[num] = X_imgs[image][j+y][i+x]
                    else:
                        filtered[num] = max_pooled_img[j+y][i+x]
                    num += 1



    filtered_im = np.zeros(convolve_flat//kernel)

    for i in range(convolve_flat//kernel):
            filtered_im[i] = np.dot(concat_weights, filtered[(i*kernel):(kernel + (i*kernel))])

    filtered_im = filtered_im.reshape(new_img_res, new_img_res)
    #filtered_im /= 255
    #plt.imshow(filtered_im, cmap="gray")
    #plt.show()
        
    return filtered_im, new_img_res #new_img_res, filtered_im


def mock_max_pooling(max_pool_side, img_res, filtered_im):
    
    
    #plt.figure(figsize = (3,3))
    max_pool = max_pool_side**2

    max_pools_per_side = img_res//max_pool_side
    max_pools_tot = max_pools_per_side**2

    max_pooled_arr = np.zeros(max_pool*(img_res//max_pool_side)**2)
    max_pooled = np.zeros(max_pools_tot)

    num = 0
    for j in range(max_pools_per_side):
        for i in range(max_pools_per_side):
            for y in range(max_pool_side):
                for x in range(max_pool_side):
                    max_pooled_arr[num] = filtered_im[(j*max_pool_side)+y][(i*max_pool_side)+x]
                    num += 1

    for pixels in range(max_pools_tot):
        max_pooled[pixels] = max(max_pooled_arr[(max_pool*pixels):(max_pool+max_pool*pixels)])

    max_pool_img = max_pooled.reshape(max_pools_per_side, max_pools_per_side)
    #plt.imshow(max_pool_img, cmap="gray")
    #plt.show()

    return max_pool_img, max_pools_per_side

In [ ]:
filter_num = 3
image = 1502

plt.figure(figsize = (20,20))

filtered_im, new_img_res = mock_convolutional(layer_num=0,
                                   filter_num=filter_num,
                                   conv_kernel_side=kernel_sides[0],
                                   img_res=img_res,
                                   image=image)
plt.subplot(1,8,1)
plt.imshow(filtered_im,cmap="gray")


max_pool_img, max_pools_per_side = mock_max_pooling(max_pool_side=max_pool_sides[0],
                                                     img_res=new_img_res,
                                                     filtered_im=filtered_im)
plt.subplot(1,8,2)
plt.imshow(max_pool_img,cmap="gray")


filtered_im, new_img_res = mock_convolutional(layer_num=3,
                                              filter_num=0,
                                              conv_kernel_side=kernel_sides[1],
                                              img_res=new_img_res,
                                              max_prev = True,
                                              max_pooled_img=max_pool_img,
                                              max_pools_per_side=max_pools_per_side)
plt.subplot(1,8,3)
plt.imshow(filtered_im,cmap="gray")


max_pool_img, max_pools_per_side = mock_max_pooling(max_pool_side=max_pool_sides[1],
                                                     img_res=new_img_res,
                                                     filtered_im=filtered_im)
plt.subplot(1,8,4)
plt.imshow(max_pool_img,cmap="gray")


filtered_im, new_img_res = mock_convolutional(layer_num=6, 
                                              filter_num=0, 
                                              conv_kernel_side=kernel_sides[2], 
                                              img_res=new_img_res,
                                              max_prev = True, 
                                              max_pooled_img=max_pool_img,
                                              max_pools_per_side=max_pools_per_side)
plt.subplot(1,8,5)
plt.imshow(filtered_im,cmap="gray")


max_pool_img, max_pools_per_side = mock_max_pooling(max_pool_side=max_pool_sides[2],
                                                     img_res=new_img_res,
                                                     filtered_im=filtered_im)
plt.subplot(1,8,6)
plt.imshow(max_pool_img,cmap="gray")


filtered_im, new_img_res = mock_convolutional(layer_num=9, 
                                              filter_num=0, 
                                              conv_kernel_side=kernel_sides[3], 
                                              img_res=new_img_res,
                                              image = 0, 
                                              max_prev = True, 
                                              max_pooled_img=max_pool_img,
                                              max_pools_per_side=max_pools_per_side)
plt.subplot(1,8,7)
plt.imshow(filtered_im,cmap="gray")